In [26]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import re


In [27]:
pd.set_option('display.max_columns',40)
options = webdriver.ChromeOptions()
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument("--disable-gpu")

df=pd.DataFrame()
Compound_Name=[]
Attibute_Name=[]
Custome_Value=[]
tille_index=[]
pattern = r'\d+\.\s(.*)'
timeout=10
url = f'https://vinmec.com/vi/thong-tin-duoc/su-dung-thuoc-toan/?page='


In [28]:

service = Service('/ChromeDriver/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome(service=service,options=options)
def get_Url(url,index_page):
    while True:
        try:
            url_page =url+index_page
            driver.get(url_page)
            WebDriverWait(driver,timeout).until(EC.text_to_be_present_in_element((By.XPATH,'/html/body/div[2]/div/div[2]'),'Sử dụng thuốc an toàn'))
            div_content=driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[4]')
            ul_tags=div_content.find_element(By.TAG_NAME,'ul')
            li_tags=ul_tags.find_elements(By.TAG_NAME,'li')
            break
        except TimeoutException:
            driver.refresh()
    return li_tags



In [29]:
def get_List_Title(drug_body_text):
    drug_body_line=drug_body_text.split('\n')
    pattern = r'\d+\.\s(.*)'
    list_title=[]
    for i,drug in enumerate(drug_body_line):
        title=re.findall(pattern,drug)
        if title:
            list_title.append(title)
    return list_title

In [30]:
def get_List_Title_Index(drug_body_text):
    drug_body_line=drug_body_text.split('\n')
    pattern = r'\d+\.\s(.*)'
    tille_index=[]
    for i,drug in enumerate(drug_body_line):
        title=re.findall(pattern,drug)
        if title:
            tille_index.append(i)
    return tille_index

In [31]:
def get_Drug_Content(drug_body_text,tille_index):
    drug_body_line=drug_body_text.split('\n')
    string_text=''
    for i in range(0,len(tille_index)):
        index=tille_index[i]
        for j in range(index,len(drug_body_line)):
            if(tille_index[-1]!=tille_index[i]):
                if(j==tille_index[i]):
                    my_string = ', '.join(str(j) for j in drug_body_line[j+1:tille_index[i+1]])
                    string_text=string_text+my_string+'\n'
            else:
                my_string = ', '.join(str(j) for j in drug_body_line[j+1:len(drug_body_line)])
                string_text=string_text+my_string
                break
                
    Array_temp=string_text.split('\n')
    return(Array_temp)

In [32]:
for j in range(250,350):
    li_tags=get_Url(url,str(j))
    list_name=[]
    i=1
    while i<= len(li_tags):
        list_name=[]
        try:
            items = driver.find_element(By.XPATH,'/html/body/div[2]/div/div[2]/div[4]/ul/li[{}]/div/h2/a'.format(i)).click()
            time.sleep(3)
            header=driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div[2]/div/div[2]/div[2]')
            h1=header.find_element(By.TAG_NAME,'h1').text
            
            drug_body_all=driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div[2]/div/div[2]')
            try:
                button=drug_body_all.find_element(By.ID,'readmore_content')
                button.click()
              
                time.sleep(2)
             
            except:
               
                pass
            
            drug_body=driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div[2]/div/div[2]/div[3]/div[1]/div[1]')
        
            try:
                list_title_index=get_List_Title_Index(drug_body.text)
                list_title=get_List_Title(drug_body.text)
                list_drug=get_Drug_Content(drug_body.text,list_title_index)
                list_name[len(list_name):] = [h1] * len(list_title) 
                for name in list_name:
                    Compound_Name.append(name)
                for title in list_title:
                    Attibute_Name.append(title)
                for drug in list_drug:
                    Custome_Value.append(drug)
                if (len(Custome_Value) > len(Attibute_Name)):
                    print('Page Content lớn hơn :',j,'\n',h1)
                    Compound_Name.append(None)
                    Attibute_Name.append(None)
            except Exception :
                print(Exception)
            pass
            driver.back()
            time.sleep(4)
            i+=1
        except:
            driver.get(url+str(j))
            time.sleep(4)
    

Page Content lớn hơn : 250 
 Gastosic và một số lưu ý khi sử dụng
Page Content lớn hơn : 250 
 Kiện Cốt Vương: Công dụng và cách sử dụng
Page Content lớn hơn : 252 
 Kem dưỡng ẩm Esunvy tăng cường độ ẩm, dưỡng da mịn màng


In [33]:
print('Độ dài của list name:',len(Compound_Name))
print('Độ dài của list Attribute:',len(Attibute_Name))
print('Độ dài của list content:',len(Custome_Value))

Độ dài của list name: 6064
Độ dài của list Attribute: 6064
Độ dài của list content: 6064


In [21]:
Compound_Name.append(None)
Attibute_Name.append(None)

In [23]:
df['Compound_Name']=Compound_Name
df['Attibute_Name']=Attibute_Name
df['Content_Value']=Custome_Value

In [24]:
df

,Compound_Name,Attibute_Name,Content_Value
0,Công dụng thuốc Chirocaine,[Chirocaine là thuốc gì?],Thuốc Chirocaine chứa thành phần Levobupivacai...
1,Công dụng thuốc Chirocaine,[Chỉ định và chống chỉ định sử dụng thuốc Chir...,Thuốc Chirocaine được chỉ định sử dụng ca cả n...
2,Công dụng thuốc Chirocaine,[Liều lượng và cách sử dụng thuốc Chirocaine],"Thuốc Chirocaine được sử dụng ở dạng tiêm, trá..."
3,Công dụng thuốc Chirocaine,[Tác dụng phụ khi sử dụng thuốc Chirocaine],Thuốc Chirocaine có thể gây ra một số tác dụng...
4,Công dụng thuốc Dongetran,[Thuốc Dongetran có tác dụng gì?],
...,...,...,...
14998,Công dụng thuốc Arclenxyl,[Tác dụng phụ của thuốc Arclenxyl],Tốt nhất vẫn là uống thuốc đúng thời gian bác ...
14999,Công dụng thuốc Arclenxyl,[Tương tác thuốc Arclenxyl],Triệu chứng quá liều ở người lớn khi dùng thuố...
15000,None,None,Thận trọng khi dùng thuốc Arclenxyl với người ...
15001,None,None,Khi sử dụng Arclenxyl với liều lớn hơn 10 mg m...


In [25]:
df.to_csv('/FRT/16.Side_Effect/4.Crawl_data_Vinmec/Thuoc/250_350.csv')